# Orizon: Raio-X

Vamos entender como os serviços se relacionam entre si computando a frequencia relativa à cada um deles por ID conta e a frequência de agrupamentos de serviços por ID conta.

Será utilizado Regras de Associação e o agrupamento feito pelo algoritmo baseado no Eclat(Equivalent Class Transformation).

In [2]:
import findspark
findspark.init("/home/raven/.spark")

In [45]:
from pyspark.sql import SparkSession, SQLContext
from pyspark.sql.functions import udf
from pyspark.sql.types import FloatType, IntegerType
import numpy as np

Inicializando sessão do Spark:

In [52]:
spark = SparkSession.builder \
   .master("local[4]") \
   .appName("Orizon - Raio X") \
   .config("spark.executor.memory", "1gb") \
   .getOrCreate()
sc = spark.sparkContext
sqlContext= SQLContext(sc, sparkSession=spark)

In [53]:
df = sqlContext.read.format('csv').options(header='true', inferSchema='true', delimiter='\t').load('data_30735068.csv')

In [54]:
df.printSchema()

root
 |-- categoria: string (nullable = true)
 |-- id_cliente: integer (nullable = true)
 |-- cliente: string (nullable = true)
 |-- id_lote: integer (nullable = true)
 |-- id_conta: integer (nullable = true)
 |-- guia_prestador: string (nullable = true)
 |-- id_item: integer (nullable = true)
 |-- id_prestador: integer (nullable = true)
 |-- prestador: string (nullable = true)
 |-- cnpj: long (nullable = true)
 |-- uf_prestador: string (nullable = true)
 |-- cidade_prestador: string (nullable = true)
 |-- id_beneficiario: string (nullable = true)
 |-- sexo: string (nullable = true)
 |-- data_nascimento: string (nullable = true)
 |-- status: string (nullable = true)
 |-- cid: string (nullable = true)
 |-- cid2: string (nullable = true)
 |-- cid3: string (nullable = true)
 |-- cid4: string (nullable = true)
 |-- senha: string (nullable = true)
 |-- crm_solicitante: string (nullable = true)
 |-- uf_crm_solicitante: string (nullable = true)
 |-- cbos_solicitante: string (nullable = true)


Selecionando apenas os campos que vamos utilizar:

In [55]:
selected_df = df.select("id_conta", "servico", "descricao_despesa", "tipo_item", "qtde", "valor")
del df
selected_df.printSchema()

root
 |-- id_conta: integer (nullable = true)
 |-- servico: string (nullable = true)
 |-- descricao_despesa: string (nullable = true)
 |-- tipo_item: string (nullable = true)
 |-- qtde: double (nullable = true)
 |-- valor: double (nullable = true)



Preparando o dataset:

In [56]:
def numeric_convert(item):
    try:
        return int(item)
    except ValueError: # replaces non convertible strings with NaN
        return np.nan

In [66]:
numeric_udf = udf(numeric_convert, IntegerType())
selected_df = selected_df.withColumn("servico", numeric_udf("servico")).dropDuplicates().na.drop()

In [58]:
selected_df.printSchema()

root
 |-- id_conta: integer (nullable = true)
 |-- servico: integer (nullable = true)
 |-- descricao_despesa: string (nullable = true)
 |-- tipo_item: string (nullable = true)
 |-- qtde: double (nullable = true)
 |-- valor: double (nullable = true)



In [59]:
selected_df.first()

Row(id_conta=1133618327, servico=70799172, descricao_despesa='ESPARADRAPO 10 X 4,5 M - B', tipo_item='MATERIAIS', qtde=120.0, valor=1.2)

Pronto, os dados parecem estar em ordem. Nesse moment, vamos trabalhar apenas com os 'id_conta' atrelados a mais de um serviço: 

In [69]:
count = selected_df.groupBy("id_conta").count()

In [76]:
count.head(10)

[Row(id_conta=1184282323, count=100),
 Row(id_conta=1184299571, count=74),
 Row(id_conta=1190390629, count=117),
 Row(id_conta=1196940285, count=100),
 Row(id_conta=1199011816, count=120),
 Row(id_conta=1208434177, count=81),
 Row(id_conta=1217489232, count=6),
 Row(id_conta=1221638720, count=62),
 Row(id_conta=1230273877, count=90),
 Row(id_conta=1245792650, count=65)]

In [95]:
selected_df = selected_df.join(selected_df.groupBy('id_conta').count(),on='id_conta')
selected_df = selected_df.filter(selected_df["count"] > 1).drop("count")

### Início da implementação do algoritmo Eclat de Regra de Associação: